In [ ]:
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Some constants 
INPUT_FOLDER = '../input/sample_images/'
patients = os.listdir(INPUT_FOLDER)
patients.sort()
patients = [k for k in patients if '.DS_Store' not in k]

# Function to get the slice thinkness
def load_scan(path):
    dicoms=os.listdir(path)
    dicoms = [k for k in dicoms if '.DS_Store' not in k]
    slices = [dicom.read_file(path + '/' + s) for s in dicoms]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

# Convert to Hounsfield Units (HU)
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)


first_patient = load_scan(INPUT_FOLDER + patients[0])
first_patient_pixels = get_pixels_hu(first_patient)
plt.hist(first_patient_pixels.flatten(), bins=80, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

# Show some slice in the middle
plt.imshow(first_patient_pixels[80], cmap=plt.cm.gray)
plt.show()


# 
path=INPUT_FOLDER + patients[0]
dicoms=os.listdir(path)
dicoms = [k for k in dicoms if '.DS_Store' not in k]
slices = [dicom.read_file(path + '/' + s) for s in dicoms]
slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
image = np.stack([s.pixel_array for s in slices])
plt.imshow(image[80], cmap=plt.cm.gray)
plt.show()

#A scan may have a pixel spacing of [2.5, 0.5, 0.5], which means that the distance between slices is 2.5 millimeters. For a different scan this may be [1.5, 0.725, 0.725], this can be problematic for automatic analysis (e.g. using ConvNets)!
#A common method of dealing with this is resampling the full dataset to a certain isotropic resolution. If we choose to resample everything to 1mm1mm1mm pixels we can use 3D convnets without worrying about learning zoom/slice thickness invariance.
#Whilst this may seem like a very simple step, it has quite some edge cases due to rounding. Also, it takes quite a while.
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array([scan[0].SliceThickness] + scan[0].PixelSpacing, dtype=np.float32)
    print ('original spacing:',spacing)

    resize_factor = spacing / new_spacing
    print ('resize_factor:',resize_factor)

    print ('image shape',image.shape)
    
    new_real_shape = image.shape * resize_factor
    print ('new real shape:',new_real_shape)
    new_shape = np.round(new_real_shape)
    print ('rounded shape:',new_shape)
    real_resize_factor = new_shape / image.shape
    print ('real size factor',real_resize_factor)
    new_spacing = spacing / real_resize_factor
    print ('new spacing',new_spacing)
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing


def plot_3d(image, threshold=-300):
    
    # Position the scan upright, 
    # so the head of the patient would be at the top facing the camera
    p = image.transpose(2,1,0)
    
    verts, faces = measure.marching_cubes(p, threshold)

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], alpha=0.70)
    face_color = [0.45, 0.45, 0.75]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])

    plt.show()

pix_resampled, spacing = resample(first_patient_pixels, first_patient, [1,1,1])
print("Shape before resampling\t", first_patient_pixels.shape)
print("Shape after resampling\t", pix_resampled.shape)

def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)

    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None
    
def segment_lung_mask(image, fill_lung_structures=True):
    
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = np.array(image > -320, dtype=np.int8)+1
    labels = measure.label(binary_image)
    
    # Pick the pixel in the very corner to determine which label is air.
    #   Improvement: Pick multiple background labels from around the patient
    #   More resistant to "trays" on which the patient lays cutting the air 
    #   around the person in half
    background_label = labels[0,0,0]
    
    #Fill the air around the person
    binary_image[background_label == labels] = 2
    
    
    # Method of filling the lung structures (that is superior to something like 
    # morphological closing)
    if fill_lung_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
            
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1

    
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1
    
    # Remove other air pockets insided body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0
 
    return binary_image

#plot_3d(pix_resampled, 400)

segmented_lungs = segment_lung_mask(pix_resampled, False)
segmented_lungs_fill = segment_lung_mask(pix_resampled, True)

In [ ]:
plot_3d(segmented_lungs, 0)

In [ ]:
np.unique(segmented_lungs[30],return_counts=True)

In [ ]:
plot_3d(segmented_lungs_fill, 0)

In [ ]:
plot_3d(segmented_lungs_fill - segmented_lungs, 0)

In [ ]:
binary_image -= 1

In [ ]:
image=pix_resampled
d=pd.DataFrame((np.array(image > -320, dtype=np.int8)+1).flatten())
#np.array(image > -320, dtype=np.int8)+1


In [ ]:
d.columns=['val']
my_tab = pd.crosstab(index=d['val'],  # Make a crosstab
                             columns='count')
my_tab

In [ ]:
binary_image = np.array(image > -320, dtype=np.int8)+1
d=pd.DataFrame(binary_image[0].flatten())
d.columns=['val']
my_tab = pd.crosstab(index=d['val'],  # Make a crosstab
                             columns='count')
my_tab
#type(d)

In [ ]:
labels = measure.label(binary_image)

In [ ]:
 background_label = labels[0,0,0]

In [ ]:
binary_image[background_label == labels] = 2
np.unique(binary_image[0],return_counts=True)

In [ ]:
axial_slice=binary_image[0]-1
labeling = measure.label(axial_slice)
np.unique(labeling,return_counts=True)

In [ ]:

axial_slice=binary_image[0]-1
labeling = measure.label(axial_slice)
im=labeling
vals, counts = np.unique(im, return_counts=True)
vals,counts
#vals = vals[vals != bg]
vals != bg

In [ ]:
for i, axial_slice in enumerate(binary_image):
    print(i)
    print(axial_slice.shape)